In [ ]:
#Import package
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,LSTM
from keras.layers import Flatten
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [ ]:
#Load pre-processed data
dataframe = pd.read_csv("timesteps=2_NeuralNetwork_new.csv")
dataframe.head()

In [194]:
#Split to training and testing data by proportion if testing = 0.2
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx,test_idx=next(gss.split(dataframe,groups=dataframe['stay_id']))


In [195]:
#get x and y variables for training and testing data
train_data=dataframe.iloc[train_idx] # data
y_train=train_data['vent_duration'] # label

x_train=train_data.drop(['Unnamed: 0','stay_id','vent_duration'],axis=1) # data
scaler=preprocessing.StandardScaler()
x_train=scaler.fit_transform(x_train)

#
test_data=dataframe.iloc[test_idx] # data
y_test=test_data['vent_duration'] # label

x_test=test_data.drop(['Unnamed: 0','stay_id','vent_duration'],axis=1) # data
scaler=preprocessing.StandardScaler()
x_test=scaler.fit_transform(x_test)


In [ ]:
#FNN model building
model = Sequential()
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse',optimizer='adam', metrics=['mse', 'mae'])

model.fit(x_train,y_train, epochs=20,  batch_size = 20, verbose=0)


In [ ]:
predictions=model.predict(x_test)

In [160]:
predictions1 = np.mean(predictions, axis=1).reshape(-1, 1)

In [ ]:
# Calculate R-squared
r2 = r2_score(y_test, predictions1)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions1))

# Calculate MSE
mse = mean_squared_error(y_test, predictions1)

# Calculate MAE
mae = mean_absolute_error(y_test, predictions1)

# Print the evaluation metrics
print('R-squared:', r2)
print('RMSE:', rmse)
print('MSE:', mse)
print('MAE:', mae)